In [2]:
!pip install -qq --upgrade pip
!pip install -qq --upgrade peft transformers accelerate bitsandbytes datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import Dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct", cache_dir="./cache")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [4]:
chat = [
    # {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
    {"role": "user", "content": "I'd like to show off how chat templating works!"},
    {"role": "assistant", "content": "Sure! What would you like me to say?"}
]

chat_prompt = tokenizer.apply_chat_template(chat, tokenize=False)
print(chat_prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 22 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Hello, how are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm doing great. How can I help you today?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'd like to show off how chat templating works!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Sure! What would you like me to say?<|eot_id|>


In [5]:
chats = [
    [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
        {"role": "user", "content": "I'd like to show off how chat templating works!"},
        {"role": "assistant", "content": "Sure! What would you like me to say?"}
    ],
    [
        {"role": "user", "content": "Which is bigger, the moon or the sun?"},
        {"role": "assistant", "content": "The sun."}
    ],
    [
        {"role": "user", "content": "Which is bigger, a virus or a bacterium?"},
        {"role": "assistant", "content": "A bacterium."}
    ],
]

dataset = Dataset.from_dict({"chats": chats})
dataset

Dataset({
    features: ['chats'],
    num_rows: 3
})

In [6]:
formatted_dataset = dataset.map(lambda x: {"formatted_chat": tokenizer.apply_chat_template(x["chats"], tokenize=False, add_generation_prompt=False, add_special_tokens=False)})
print(formatted_dataset['formatted_chat'])

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 22 Feb 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHello, how are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI'm doing great. How can I help you today?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI'd like to show off how chat templating works!<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSure! What would you like me to say?<|eot_id|>", '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 22 Feb 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhich is bigger, the moon or the sun?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe sun.<|eot_id|>', '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 22 Feb 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nW

In [7]:
messages_field = "chats"  # Replace with the actual field name
def format_dataset(examples):
    if isinstance(examples[messages_field][0], list):
        output_texts = []
        for i in range(len(examples[messages_field])):
            output_texts.append(tokenizer.apply_chat_template(examples[messages_field][i], tokenize=False))
        return {"formatted_text": output_texts}
    else:
        return {"formatted_text": tokenizer.apply_chat_template(examples[messages_field], tokenize=False)}

formatted_dataset = dataset.map(format_dataset)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [8]:
tokenized_sample = tokenizer(formatted_dataset['formatted_text'][0], padding=True, truncation=True, return_special_tokens_mask=True, add_special_tokens=False)
tokenized_sample,

({'input_ids': [128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1313, 13806, 220, 2366, 20, 271, 128009, 128006, 882, 128007, 271, 9906, 11, 1268, 527, 499, 30, 128009, 128006, 78191, 128007, 271, 40, 2846, 3815, 2294, 13, 2650, 649, 358, 1520, 499, 3432, 30, 128009, 128006, 882, 128007, 271, 40, 4265, 1093, 311, 1501, 1022, 1268, 6369, 68630, 1113, 4375, 0, 128009, 128006, 78191, 128007, 271, 40914, 0, 3639, 1053, 499, 1093, 757, 311, 2019, 30, 128009], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [9]:
tokenizer.decode(tokenized_sample['input_ids'], skip_special_tokens=False)

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 22 Feb 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHello, how are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI'm doing great. How can I help you today?<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI'd like to show off how chat templating works!<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSure! What would you like me to say?<|eot_id|>"

## References

- https://huggingface.co/docs/transformers/main/en/chat_templating
- https://github.com/huggingface/trl/blob/main/trl/trainer/utils.py#L65
- https://gist.github.com/Blaizzy/40de0f6b4340490e3920db9e182e6455
- https://www.kaggle.com/code/lonnieqin/instruction-tuning-with-llm
- https://openreview.net/pdf?id=GcZgo9ffGt


In [11]:
from datasets import load_dataset

tmp_dataset = load_dataset("trl-lib/Capybara", split="train")
tmp_dataset, tmp_dataset[0]

README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/37.2M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/482k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15806 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

(Dataset({
     features: ['source', 'messages', 'num_turns'],
     num_rows: 15806
 }),
 {'source': 'GPT4LLM',
  'messages': [{'content': 'Recommend a movie to watch.\n', 'role': 'user'},
   {'content': 'I would recommend the movie, "The Shawshank Redemption" which is a classic drama film starring Tim Robbins and Morgan Freeman. This film tells a powerful story about hope and resilience, as it follows the story of a young man who is wrongfully convicted of murder and sent to prison. Amidst the harsh realities of prison life, the protagonist forms a bond with a fellow inmate, and together they navigate the challenges of incarceration, while holding on to the hope of eventual freedom. This timeless movie is a must-watch for its moving performances, uplifting message, and unforgettable storytelling.',
    'role': 'assistant'},
   {'content': 'Describe the character development of Tim Robbins\' character in "The Shawshank Redemption".',
    'role': 'user'},
   {'content': 'In "The Shawsha